# Decision Tree

In [1]:
using MLJ
using RDatasets

┌ Info: Precompiling MLJ [add582a8-e3ab-11e8-2d5e-e98b27df1bc7]
└ @ Base loading.jl:1423
┌ Warning: Replacing module `CategoricalDistributions`
└ @ Base loading.jl:1036
┌ Warning: Module MLJBase with build ID 106585755292894 is missing from the cache.
│ This may mean MLJBase [a7f614a8-145f-11e9-1d2a-a57a1082229d] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1107
┌ Warning: The call to compilecache failed to create a usable precompiled cache file for MLJ [add582a8-e3ab-11e8-2d5e-e98b27df1bc7]
│   exception = ErrorException("Required dependency MLJTuning [03970b2e-30c4-11ea-3135-d1576263f10f] failed to load from a cache file.")
└ @ Base loading.jl:1132
┌ Warning: Replacing module `CategoricalDistributions`
└ @ Base loading.jl:1036
┌ Info: Precompiling MLJIteration [614be32b-d00c-4edb-bd02-1eb411ab5e55]
└ @ Base loading.jl:1423
┌ Warning: Module MLJBase with build ID 106585755292894 is missing from the cache.
│ This may mean MLJBase [a7f614a8-

In [2]:
smarket = dataset("ISLR", "Smarket")
first(smarket, 6)

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Cat…
1,2001.0,0.381,-0.192,-2.624,-1.055,5.01,1.1913,0.959,Up
2,2001.0,0.959,0.381,-0.192,-2.624,-1.055,1.2965,1.032,Up
3,2001.0,1.032,0.959,0.381,-0.192,-2.624,1.4112,-0.623,Down
4,2001.0,-0.623,1.032,0.959,0.381,-0.192,1.276,0.614,Up
5,2001.0,0.614,-0.623,1.032,0.959,0.381,1.2057,0.213,Up
6,2001.0,0.213,0.614,-0.623,1.032,0.959,1.3491,1.392,Up


In [3]:
y, X = unpack(smarket, ==(:Direction), colname -> true);
X = select(X, Not([:Year, :Today]))
first(X, 6)

,Lag1,Lag2,Lag3,Lag4,Lag5,Volume
,Float64,Float64,Float64,Float64,Float64,Float64
1,0.381,-0.192,-2.624,-1.055,5.01,1.1913
2,0.959,0.381,-0.192,-2.624,-1.055,1.2965
3,1.032,0.959,0.381,-0.192,-2.624,1.4112
4,-0.623,1.032,0.959,0.381,-0.192,1.276
5,0.614,-0.623,1.032,0.959,0.381,1.2057
6,0.213,0.614,-0.623,1.032,0.959,1.3491


In [4]:
y = coerce(y, OrderedFactor)
classes(y[1])

2-element CategoricalArrays.CategoricalArray{String,1,UInt8}:
 "Down"
 "Up"

### Training/testing set

In [5]:
train, test = partition(eachindex(y), 0.7, shuffle=true)

([183, 753, 949, 470, 668, 725, 453, 908, 1070, 896  …  670, 985, 616, 719, 375, 36, 61, 1008, 599, 191], [877, 686, 1040, 165, 465, 337, 421, 878, 879, 334  …  443, 592, 893, 507, 964, 857, 4, 775, 635, 352])

### Model

In [6]:
DecisionTreeClassifier = @load DecisionTreeClassifier pkg=DecisionTree

import MLJDecisionTreeInterface ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /home/yuehhua/.julia/packages/MLJModels/lDzCR/src/loading.jl:168


MLJDecisionTreeInterface.DecisionTreeClassifier

In [7]:
tree = machine(DecisionTreeClassifier(), X, y)

Machine trained 0 times; caches data
  model: DecisionTreeClassifier(max_depth = -1, …)
  args: 
    1:	Source @895 ⏎ `Table{AbstractVector{Continuous}}`
    2:	Source @590 ⏎ `AbstractVector{OrderedFactor{2}}`


### Training

In [8]:
MLJ.fit!(tree, rows=train)

┌ Info: Training machine(DecisionTreeClassifier(max_depth = -1, …), …).
└ @ MLJBase /home/yuehhua/.julia/packages/MLJBase/rQDaq/src/machines.jl:487


Machine trained 1 time; caches data
  model: DecisionTreeClassifier(max_depth = -1, …)
  args: 
    1:	Source @895 ⏎ `Table{AbstractVector{Continuous}}`
    2:	Source @590 ⏎ `AbstractVector{OrderedFactor{2}}`


### Prediction

In [9]:
ŷ = predict_mode(tree, rows=test)

375-element CategoricalArrays.CategoricalArray{String,1,UInt8}:
 "Up"
 "Up"
 "Up"
 "Up"
 "Down"
 "Down"
 "Up"
 "Down"
 "Up"
 "Down"
 "Down"
 "Up"
 "Up"
 ⋮
 "Up"
 "Up"
 "Down"
 "Down"
 "Down"
 "Down"
 "Down"
 "Up"
 "Up"
 "Down"
 "Down"
 "Down"

## Evaluation

In [10]:
accuracy(ŷ, y[test])

0.5066666666666666